In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# GPU check
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set memory growth for GPU to avoid full memory allocation
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("No GPU available, using CPU.")

In [ ]:
# Load datasets
DATA_PATH = 'D:/NTHU_NLP_2024_Term_Project_35/'
train_essays = pd.read_csv(f'{DATA_PATH}/train_essays.csv')
prompts = pd.read_csv(f'{DATA_PATH}/train_prompts.csv')
train_v2 = pd.read_csv(f'{DATA_PATH}/train_v2_drcat_02.csv')
train_lim = pd.read_csv(f'{DATA_PATH}/ai_generated_train_essays.csv')
train_lim2 = pd.read_csv(f'{DATA_PATH}/ai_generated_train_essays_gpt-4.csv')

In [ ]:
# Combine datasets
combined_from_comp = pd.merge(train_essays, prompts, on='prompt_id', how='left')
train_lim = pd.concat([train_lim, train_lim2], ignore_index=True)
train_merged = pd.concat([combined_from_comp, train_lim], ignore_index=True)

In [ ]:
# Preprocessing function
def remove_tag(text):
    tag = re.compile(r'@\S+')
    return tag.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(url, '', text)

def remove_html(text):
    html = re.compile(r'<[^>]+>|\([^)]+\)')
    return html.sub(r'', text)

def remove_punct(text):
    punctuations = list(string.punctuation)
    table = str.maketrans('', '', ''.join(punctuations))
    return text.translate(table)

In [ ]:
# Apply cleaning
train_merged['cleaned'] = train_merged['text'].apply(lambda x: remove_tag(x))
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: remove_URL(x))
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: remove_html(x))
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: remove_punct(x))
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: x.lower())
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: nltk.word_tokenize(x))
stopwords = set(nltk.corpus.stopwords.words('english'))
train_merged['cleaned'] = train_merged['cleaned'].apply(lambda x: ' '.join([word for word in x if word not in stopwords]))

In [ ]:
# Balancing dataset
majority_class = train_merged[train_merged['generated'] == 0]
minority_class = train_merged[train_merged['generated'] == 1]
majority_downsampled = resample(majority_class, replace=False, n_samples=len(minority_class), random_state=42)
train_merged = pd.concat([majority_downsampled, minority_class])

In [ ]:
# Split dataset
train_val_df, test_df = train_test_split(train_merged, test_size=0.1, random_state=42, stratify=train_merged['generated'])
train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=42, stratify=train_val_df['generated'])

In [ ]:
# Tokenizer and encoding
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_df['cleaned'].tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_df['cleaned'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_df['cleaned'].tolist(), truncation=True, padding=True, max_length=512)

In [ ]:
# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_df['generated'].tolist()))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_df['generated'].tolist()))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_df['generated'].tolist()))

In [ ]:
# Batch datasets
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(8)
val_dataset = val_dataset.batch(16)
test_dataset = test_dataset.batch(16)

In [ ]:
# Create strategy for multi-GPU training if multiple GPUs are available
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
# Define and compile the model within strategy scope
with strategy.scope():
    base_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

    class DistilBertWithDropout(tf.keras.Model):
        def __init__(self, base_model, num_labels, dropout_rate=0.3):
            super(DistilBertWithDropout, self).__init__()
            self.base_model = base_model
            self.dropout = tf.keras.layers.Dropout(dropout_rate)
            self.classifier = tf.keras.layers.Dense(num_labels, activation='softmax')

        def call(self, inputs, training=False):
            outputs = self.base_model(inputs)
            pooled_output = outputs[0][:, 0, :]
            dropout_output = self.dropout(pooled_output, training=training)
            logits = self.classifier(dropout_output)
            return logits

    model = DistilBertWithDropout(base_model, num_labels=2)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

In [ ]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_model", save_best_only=True, save_format='tf')
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=4, callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Confusion Matrix
y_pred = model.predict(test_dataset)
y_pred_labels = tf.argmax(y_pred, axis=1)
y_true = test_df['generated'].values
cf_matrix = confusion_matrix(y_true, y_pred_labels)

In [ ]:
# Display confusion matrix
categories = ['Negative', 'Positive']
disp = ConfusionMatrixDisplay(confusion_matrix=cf_matrix, display_labels=categories)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# Save the model
model.save('AI-detector', save_format='tf')